## Lab: RNN with Forecasting

### Wind Turbine Power Prediction with RNN

This lab uses a Recurrent Neural Network to predict wind turbine power output.
* [Data link](https://www.kaggle.com/code/ahmedfathygwely/wind-turbine-dataset-machine-learning-rnn-times/input)

### Step 0: Load libraries & Data

In [ ]:
# Step 0: Load libraries & Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import math

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the data
df = pd.read_csv('T1.csv')

In [ ]:
# Display basic information


### Step 1: Prepare the data

In [ ]:
# Step 1: Prepare the data


### Step 2: Create sequences

In [ ]:
# Step 2: Create sequences



### Step 3: Split the data

In [ ]:
# Step 3: Split the data


### Step 4: Build the RNN Model

In [ ]:
# Step 4: Build the RNN Model



### Step 5: Train the model

In [ ]:
# Step 5: Train the model


### Step 6: Evaluate on the test data and visualize the results

In [ ]:
# Step 6: Evaluate and visualize